In [52]:
# Installation of libraries
!pip uninstall -y fitz
!pip uninstall -y pymupdf
!pip install pymupdf==1.23.7
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: PyMuPDF 1.23.7
Uninstalling PyMuPDF-1.23.7:
  Successfully uninstalled PyMuPDF-1.23.7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached PyMuPDF-1.23.7-cp311-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached PyMuPDF-1.23.7-cp311-none-manylinux2014_x86_64.whl (4.4 MB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 57.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.3
    Uninstalling transformers-4.55.3:
      Successfully uninstalled transformers-4.55.3


In [53]:
# Importing libraries
import os
import re
import gc
import fitz  
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import torch.optim as optim 
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    get_linear_schedule_with_warmup, set_seed
)
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, roc_auc_score,
    precision_score, recall_score, f1_score, roc_auc_score
)
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [54]:
nltk.download('punkt')

# 1) Hyperparameters
SEED   = 5
EPOCHS = 100         # 3       
LR     = 2e-5       
BATCH  = 16
MODEL  = "bert-base-uncased"
MAXLEN = 512           #128

set_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Configuration
CONFIG = {
    "max_length": MAXLEN,
    "batch_size": BATCH,
    "learning_rate": LR,
    "epochs": EPOCHS,
    "context_window": 3,  # sentences around match
    "min_context_length": 50,
    "confidence_threshold": 0.7,  
    "data_paths": {
        "train_csv": "/kaggle/working/processed_text_dataframe.csv",
        "eval_metrics": ["macro_f1", "weighted_f1", "precision", "recall"],
    }
}

FILE_PATH = '/kaggle/input/suhi-data-sheet/suhi_data.xlsx'
OUTPUT_CSV = '/kaggle/working/processed_text_dataframe.csv'
BERT_CLF_DIRECTORY = "/kaggle/working/bert_textclf_ckpt"

# Training constants
TEST_SPLIT_RATIO = 0.2
N_COMPONENT = 50

# Data constants
FETAURE_SUBSET = 2 
PATIENT_SUBSET = 2
VECTORIZE_SUBSET = 2 
ENGAGED = 1
SUMMERIZED = 0

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("Setup complete.\n")

Using device: cuda
Setup complete.



In [67]:
def load_and_process():
    # Load & filter source rows
    df = pd.read_excel(FILE_PATH)
    df = df[df['engaged'] == ENGAGED].reset_index(drop=True)

    # Keep only the two columns and rename
    needed = df[['COMBINED_NOTES', 'day_readmit']].copy()
    needed = needed.rename(columns={'COMBINED_NOTES': 'text', 'day_readmit': 'label'})

    # Clean label: drop NaN, cast to int, keep only 0/1
    needed = needed.dropna(subset=['label'])
    needed['label'] = needed['label'].astype(int)
    needed = needed[needed['label'].isin([0, 1])].reset_index(drop=True)

    # Clean text: coerce to string, drop empty/whitespace-only
    needed['text'] = needed['text'].fillna('').astype(str)
    needed = needed[needed['text'].str.strip() != ''].reset_index(drop=True)
    return needed

In [68]:
def get_dataframe_metrics(df):
    print("df shape")
    print(df.shape)
    print("-----------")

def get_unique_values(df):
    unique, counts = np.unique(df["label"], return_counts=True)
    label_counts = dict(zip(unique, counts))
    print("label_counts") 
    print(label_counts) 
    print("-----------") 

def save_new_data_text_dataframe(dataframe):
    dataframe.to_csv(OUTPUT_CSV, index = False)

In [69]:
df = load_and_process()
save_new_data_text_dataframe(df)

In [70]:
# Clean & validate your two columns
assert {'text','label'}.issubset(df.columns), "df must have 'text' and 'label'"
# keep only those two columns
df = df[['text','label']].copy()

In [71]:
# drop rows with NaN/empty text
df['text'] = df['text'].astype(str).str.strip()
df = df[df['text'] != '']
df = df.dropna(subset=['text'])

# coerce label to int and restrict to {0,1}
df['label'] = df['label'].astype(int)
assert set(df['label'].unique()).issubset({0,1}), "label must be 0/1 only"

In [72]:
def get_top_n_longest_texts(df, label_col, text_col, label_value=0, top_n=100):
    filtered_df = df[df[label_col] == label_value].copy()
    filtered_df["text_len"] = filtered_df[text_col].str.len()
    top_rows = filtered_df.sort_values("text_len", ascending=False).head(top_n)
    return top_rows.drop(columns=["text_len"])

In [73]:
top_100_0s = get_top_n_longest_texts(df, label_col="label", text_col="text", label_value=0, top_n=200)
top_100_1s = get_top_n_longest_texts(df, label_col="label", text_col="text", label_value=1, top_n=200)
combined_df = pd.concat([top_100_0s, top_100_1s], ignore_index=True)



In [74]:
print(get_unique_values(combined_df))

label_counts
{0: 200, 1: 111}
-----------
None


In [75]:
# Split train / test: 80/ 20 (stratified)
train_df, test_df = train_test_split(combined_df, test_size=TEST_SPLIT_RATIO, random_state=SEED, stratify=combined_df['label'])


# train_df, test_df = train_test_split(df, test_size=TEST_SPLIT_RATIO, random_state=SEED, stratify=combined_df['label'])

In [76]:
print(f"Train={len(train_df)}, Test={len(test_df)}")
get_dataframe_metrics(train_df)
train_df.head(5)
get_unique_values(train_df)

Train=248, Test=63
df shape
(248, 2)
-----------
label_counts
{0: 159, 1: 89}
-----------


In [77]:
train_texts = train_df["text"].tolist()
train_labels = train_df["label"].tolist()

test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()


print(len(train_texts))
print(len(train_labels))
print("------------")


print(len(test_texts))
print(len(test_labels))
print("------------")

248
248
------------
63
63
------------


In [78]:
class SUHIDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):    
        text = self.texts[index]
        label = self.labels[index]
    
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
    
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }


In [79]:
def compute_class_weights(labels):
    classes = np.unique(labels)
    weights = compute_class_weight('balanced', classes=classes, y=labels)
    return torch.tensor(weights, dtype=torch.float).to(device)

In [80]:
def fine_tune_model(train_texts, train_labels):
    # Initializing tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    ).to(device)

    dataset = SUHIDataset(train_texts, train_labels, tokenizer, CONFIG["max_length"])
    
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"]) 
    total_steps = len(loader) * CONFIG["epochs"]
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    class_weights = compute_class_weights(train_labels)
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

    # Training loop
    for epoch in range(CONFIG["epochs"]):
        model.train()
        total_loss = 0
        correct_predictions = 0

        progress_bar = tqdm(loader, desc=f"Epoch {epoch + 1}/{CONFIG['epochs']}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()

            # Progress bar
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{torch.sum(preds == labels).item()/len(labels):.2f}"
            })

        epoch_loss = total_loss / len(loader)
        epoch_acc = correct_predictions / len(dataset)
        print(f"Epoch {epoch + 1} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    return model, tokenizer

In [81]:
# Training the model
model, tokenizer = fine_tune_model(train_texts, train_labels)
print("Model training completed")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 - Loss: 0.6519, Accuracy: 0.6976


Epoch 2/100:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 2 - Loss: 0.5514, Accuracy: 0.8266


...

Epoch 98 - Loss: 0.0006, Accuracy: 1.0000


Epoch 99/100:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 99 - Loss: 0.0007, Accuracy: 1.0000


Epoch 100/100:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 100 - Loss: 0.0006, Accuracy: 1.0000
Model training completed


In [82]:
def enhanced_evaluate(model, tokenizer, texts, labels, batch_size=16, max_len=128, device='cuda'):
    dataset = SUHIDataset(texts, labels, tokenizer, max_len)
    loader = DataLoader(dataset, batch_size=batch_size)

    true_labels = []
    pred_labels = []
    pred_probs = []

    model.eval()
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)[:, 1]  # probability of class 1

            preds = torch.argmax(logits, dim=1)

            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())
            pred_probs.extend(probs.cpu().numpy())

    # Confusion matrix
    cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1])
    print("Confusion Matrix:\n", cm)

    # Classification report
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, labels=[0, 1], target_names=["0", "1"], digits=4))

    # Macro averages
    macro_avg = {
        "Precision": precision_score(true_labels, pred_labels, average="macro"),
        "Recall": recall_score(true_labels, pred_labels, average="macro"),
        "F1": f1_score(true_labels, pred_labels, average="macro")
    }

    # AUC score
    try:
        auc = roc_auc_score(true_labels, pred_probs)
        print(f"\n AUC Value: {auc:.4f}")
        macro_avg["AUC"] = auc
    except ValueError:
        macro_avg["AUC"] = None

    print("\nMacro Averages:")
    for k, v in macro_avg.items():
        print(f"{k}: {v:.4f}" if v is not None else f"{k}: N/A")

    if cm.shape != (2, 2):
        raise ValueError("Confusion matrix must be 2x2 for binary classification.")
    tn, fp, fn, tp = cm.ravel()
    print(f"TP: {tp} TN: {tn} FP:  {fp}  FN:  {fn}")
    
    return cm, macro_avg


In [83]:
test_cm, test_macro = enhanced_evaluate(model, tokenizer, test_texts, test_labels, device=device)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Confusion Matrix:
 [[ 9 32]
 [ 2 20]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8182    0.2195    0.3462        41
           1     0.3846    0.9091    0.5405        22

    accuracy                         0.4603        63
   macro avg     0.6014    0.5643    0.4433        63
weighted avg     0.6668    0.4603    0.4140        63


 AUC Value: 0.6308

Macro Averages:
Precision: 0.6014
Recall: 0.5643
F1: 0.4433
AUC: 0.6308
TP: 20 TN: 9 FP:  32  FN:  2
